In [19]:
!pip install razdel datasets transformers seqeval accelerate plac

Подгружаем данные

In [4]:
import pandas as pd
data = pd.read_csv("ner_data_train.csv")
data

,video_info,entities
0,<НАЗВАНИЕ:> Агент 117: Из Африки с любовью — Р...,"{""label"":""локация""\,""offset"":26\,""length"":6\,""..."
1,<НАЗВАНИЕ:> Коленвал Инфинити Ку икс 56= 5.6 V...,"{""label"":""организация""\,""offset"":196\,""length""..."
2,<НАЗВАНИЕ:> ВЫЗОВ ДЕМОНА = Вызвал Серого Челов...,"{""label"":""название проекта""\,""offset"":12\,""len..."
3,<НАЗВАНИЕ:> Довоенная немецкая кирха в Калинин...,"{""label"":""не найдено""\,""offset"":162\,""length"":..."
4,"<НАЗВАНИЕ:> ""Спартаку"" помогли судьи? Локомоти...","{""label"":""команда""\,""offset"":13\,""length"":8\,""..."
...,...,...
6417,<НАЗВАНИЕ:> ЗАПРЕЩЕННЫЙ УДАР! Полный бой Вадим...,"{""label"":""персона""\,""offset"":41\,""length"":12\,..."
6418,<НАЗВАНИЕ:> Как создаются медали: оптические и...,"{""label"":""локация""\,""offset"":88\,""length"":10\,..."
6419,"<НАЗВАНИЕ:> Ремонт ломовоза камаз, монтаж гидр...","{""label"":""бренд""\,""offset"":28\,""length"":5\,""se..."
6420,<НАЗВАНИЕ:> Голодные игры: Баллада о змеях и п...,"{""label"":""Дата""\,""offset"":78\,""length"":4\,""seg..."


Предобработка данных

In [5]:
import json
df = data.copy()
df['entities'] = df['entities'].apply(lambda l: l.replace('\,', ',')if isinstance(l, str) else l)
df['entities'] = df['entities'].apply(lambda l: l.replace('\\\\', '\\')if isinstance(l, str) else l)
df['entities'] = df['entities'].apply(lambda l: '[' + l + ']'if isinstance(l, str) else l)
df['entities'] = df['entities'].apply(lambda l: json.loads(l)if isinstance(l, str) else l)

In [6]:
from razdel import tokenize

def extract_labels(item):

    raw_toks = list(tokenize(item['video_info']))
    words = [tok.text for tok in raw_toks]
    word_labels = ['O'] * len(raw_toks)
    char2word = [None] * len(item['video_info'])
    for i, word in enumerate(raw_toks):
        char2word[word.start:word.stop] = [i] * len(word.text)

    labels = item['entities']
    if isinstance(labels, dict):
        labels = [labels]
    if labels is not None:
        for e in labels:
            if e['label'] != 'не найдено':
                e_words = sorted({idx for idx in char2word[e['offset']:e['offset']+e['length']] if idx is not None})
                if e_words:
                    word_labels[e_words[0]] = 'B-' + e['label']
                    for idx in e_words[1:]:
                        word_labels[idx] = 'I-' + e['label']
                else:
                    continue
            else:
                continue
        return {'tokens': words, 'tags': word_labels}
    else: return {'tokens': words, 'tags': word_labels}

In [7]:
print(extract_labels(df.iloc[0]))

{'tokens': ['<', 'НАЗВАНИЕ', ':', '>', 'Агент', '117', ':', 'Из', 'Африки', 'с', 'любовью', '—', 'Русский', 'тизер', '=', 'трейлер', '(', '2021', ')', '<', 'ОПИСАНИЕ', ':', '>', 'Лучший', 'Telegram', 'канал', 'о', 'кино', '<', 'LINK', '>', 'Сотрудничество', '<', 'LINK', '>', 'Дата', 'выхода', '26', 'августа', '2021', 'Оригинальное', 'название', ':', 'OSS', '117', ':', 'Alerte', 'rouge', 'en', 'Afrique', 'noire', 'Страна', ':', 'Франция', 'Режиссер', ':', 'Николя', 'Бедос', 'Жанр', ':', 'боевик', ',', 'комедия', 'В', 'главных', 'ролях', ':', 'Жан', 'Дюжарден', ',', 'Пьер', 'Нинэ', ',', 'Мелоди', 'Каста', ',', 'Наташа', 'Линдинжер', ',', 'Владимир', 'Иорданов', ',', 'Фату', 'Н', '’', 'Диайе', ',', 'Пол', 'Уайт', 'Мир', 'изменился', '.', 'Он', 'нет', '.', 'Судьба', 'заносит', 'легендарного', 'Агента', '117', 'в', 'Африку', ',', 'где', 'горячее', 'пустыни', 'только', 'женщины', '.', 'Вооруженный', 'неиссякаемой', 'уверенностью', 'в', 'себе', 'и', 'убийственной', 'харизмой', ',', 'он', 'мож

In [8]:
from sklearn.model_selection import train_test_split
ner_data = [extract_labels(item) for i, item in df.iterrows()]
ner_train, ner_test = train_test_split(ner_data, test_size=0.2, random_state=1)

In [9]:
import pandas as pd
pd.options.display.max_colwidth = 300
data = pd.DataFrame(ner_train)

Составим датасет (словарик) со всеми словами и классом именованных сущностей

In [ ]:
dataset = {}
data_wo = {}
for i in range(0, len(data)):
    text = data.tokens[i]
    tags = data.tags[i]
    for j in range(0, len(text)):
      try:
        if dataset[text[j]]:
          pass
      except:
        if tags[j] != 'O':
            data_wo[text[j]] = tags[j]
        print(f'Word: {text[j]}, tag: {tags[j]}')
        dataset[text[j]] = tags[j]

In [11]:
dataset

{'<': 'O',
 'НАЗВАНИЕ': 'O',
 ':': 'O',
 '>': 'O',
 'ТОП': 'O',
 '=': 'O',
 '5': 'O',
 'хороших': 'O',
 'видеорегистраторов': 'O',
 'с': 'O',
 'двумя': 'O',
 'камерами': 'O',
 '.': 'O',
 'Что': 'O',
 'выбрать': 'O',
 'в': 'O',
 '2022': 'B-Дата',
 '?': 'O',
 'ОПИСАНИЕ': 'O',
 'Roadgid': 'B-Дата',
 'Blick': 'I-Дата',
 'GPS': 'I-Дата',
 'LINK': 'O',
 '4': 'O',
 'iBOX': 'O',
 'iCON': 'O',
 'LV': 'O',
 'WiFi': 'O',
 'Signature': 'O',
 'Dual': 'O',
 '3.': 'O',
 '70': 'O',
 'mai': 'O',
 'Dash': 'O',
 'Cam': 'O',
 'Pro': 'O',
 'Plus': 'O',
 '2': 'O',
 'Daocam': 'O',
 'Combo': 'O',
 'CH': 'O',
 '1': 'O',
 'RoadGid': 'O',
 'CityGo': 'O',
 '3': 'O',
 'Мы': 'O',
 'Вконтакте': 'O',
 'Telegram': 'O',
 'Наш': 'O',
 'канал': 'O',
 'на': 'O',
 'Я': 'O',
 'Дзен': 'O',
 'ТикТок': 'O',
 '/': 'O',
 'AT': 'O',
 'Сотрудничество': 'O',
 'и': 'O',
 'реклама': 'O',
 'Прямая': 'O',
 'связь': 'O',
 'автором': 'O',
 '✔': 'O',
 'Нажми': 'O',
 'колокольчик': 'O',
 '!': 'O',
 'Ставь': 'O',
 'лайк': 'O',
 ',': 'O',
 '

Сделаем классы типом int

In [12]:
label_list = ["O"] + list(set(data_wo[i] for i in data_wo))
label_map = {label: i for i, label in enumerate(label_list)}

In [13]:
label_map

{'O': 0,
 'I-модель': 1,
 'I-название проекта': 2,
 'I-локация': 3,
 'B-персона': 4,
 'I-персона': 5,
 'B-локация': 6,
 'I-команда': 7,
 'B-серия': 8,
 'B-команда': 9,
 'B-видеоигра': 10,
 'B-сезон': 11,
 'I-вид спорта': 12,
 'I-сезон': 13,
 'B-Дата': 14,
 'I-организация': 15,
 'B-организация': 16,
 'B-модель': 17,
 'I-видеоигра': 18,
 'B-лига': 19,
 'I-лига': 20,
 'I-Дата': 21,
 'B-бренд': 22,
 'B-название проекта': 23,
 'B-вид спорта': 24,
 'I-серия': 25,
 'I-бренд': 26}

In [16]:
data = []

Преобразуем в нужный нам вид

In [ ]:
for i in range(0, len(dataset)):
  data.append(tuple([list(dataset.keys())[i], {"entities": list([tuple(list([0, len(list(dataset.keys())[i]), list(dataset.values())[i]]))])}]))

In [18]:
data

[('<', {'entities': [(0, 1, 'O')]}),
 ('НАЗВАНИЕ', {'entities': [(0, 8, 'O')]}),
 (':', {'entities': [(0, 1, 'O')]}),
 ('>', {'entities': [(0, 1, 'O')]}),
 ('ТОП', {'entities': [(0, 3, 'O')]}),
 ('=', {'entities': [(0, 1, 'O')]}),
 ('5', {'entities': [(0, 1, 'O')]}),
 ('хороших', {'entities': [(0, 7, 'O')]}),
 ('видеорегистраторов', {'entities': [(0, 18, 'O')]}),
 ('с', {'entities': [(0, 1, 'O')]}),
 ('двумя', {'entities': [(0, 5, 'O')]}),
 ('камерами', {'entities': [(0, 8, 'O')]}),
 ('.', {'entities': [(0, 1, 'O')]}),
 ('Что', {'entities': [(0, 3, 'O')]}),
 ('выбрать', {'entities': [(0, 7, 'O')]}),
 ('в', {'entities': [(0, 1, 'O')]}),
 ('2022', {'entities': [(0, 4, 'B-Дата')]}),
 ('?', {'entities': [(0, 1, 'O')]}),
 ('ОПИСАНИЕ', {'entities': [(0, 8, 'O')]}),
 ('Roadgid', {'entities': [(0, 7, 'B-Дата')]}),
 ('Blick', {'entities': [(0, 5, 'I-Дата')]}),
 ('GPS', {'entities': [(0, 3, 'I-Дата')]}),
 ('LINK', {'entities': [(0, 4, 'O')]}),
 ('4', {'entities': [(0, 1, 'O')]}),
 ('iBOX', {'ent

Начинаем обучать

In [20]:
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path
import spacy
from tqdm import tqdm

In [21]:
model = None
output_dir=Path("output")
n_iter=1

In [22]:
if model is not None:
    nlp = spacy.load(model)  
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('ru')  
    print("Created blank 'ru' model")


if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe('ner', last=True)
else:
    ner = nlp.get_pipe('ner')

Created blank 'ru' model


In [23]:
from spacy.training.example import Example


examples = []
for text, annotations in data:
    example = Example.from_dict(nlp.make_doc(text), annotations)
    examples.append(example)
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # Только обучение NER
      optimizer = nlp.begin_training()
      for itn in range(n_iter):
        random.shuffle(examples)
        losses = {}
        for example in tqdm(examples):
            nlp.update(
                [example],  
                drop=0.5,  
                losses=losses)
        print(losses)


100%|██████████| 1/1 [00:00<00:00, 60.39it/s]


{'ner': 0.8333333134651184}


100%|██████████| 2/2 [00:00<00:00, 146.14it/s]


{'ner': 1.640995442867279}


100%|██████████| 3/3 [00:00<00:00, 163.84it/s]


...

{'ner': 90.31221740458967}


100%|██████████| 342/342 [00:02<00:00, 156.22it/s]


{'ner': 97.2927839888254}


 66%|██████▋   | 228/343 [00:01<00:00, 162.89it/s]


KeyboardInterrupt: 

Ищем именованные сущности

In [ ]:
for text, _ in data:
    doc = nlp(text)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])

Сохраняем нашу модельку

In [ ]:
if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)

Загружаем

In [ ]:
model = spacy.load('model_name')